In [4]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [71]:
# Load our data.
df = pd.read_csv("data/Normalized_Purchase_Card_Transactions.csv")
df_agencies = pd.read_csv("data/List_Agencies.csv")
df_vendors = pd.read_csv("data/List_Vendors.csv")

In [72]:
df_vendors

,VENDOR_NUM,VENDOR_NAME
0,0,NaN
1,1,A
2,2,AAAA
3,3,AAACE_AMERICAN_ASSOCIA
4,4,AAAGA_WORLD_CONGRESS
...,...,...
38095,38095,__MANASSAS__CAPTRI
38096,38096,__OIL_PRO_INC
38097,38097,__PENTAGON_CENTR
38098,38098,__UPPER_MARLBORO_


In [6]:
blank = np.zeros((df['AGENCY_NUM'].max() + 1, df['VENDOR_NAME_NUM'].max() + 1))

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    blank[row['AGENCY_NUM'], row['VENDOR_NAME_NUM']] += row['TRANSACTION_AMOUNT_NORM']

np.count_nonzero(blank)

  0%|          | 0/390109 [00:00<?, ?it/s]

44494

In [7]:
wordCounts = pd.DataFrame(blank)

bags = {i: df[df['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].unique() for i in df['AGENCY_NUM'].unique()}

In [25]:
bags

{34: array([37719, 21724,  4184, ..., 37241,   127, 17728]),
 43: array([ 6406, 19485, 12057, ..., 34629, 15597, 20903]),
 24: array([36174,  1390,  9816, ..., 37372, 21143, 21357]),
 59: array([32645, 11555, 32622, 35086, 30083, 20960, 21008, 16606, 33804,
        35775, 23430, 27590,  9913, 22092, 34803,  8773, 35693, 30128,
        33906, 36576,   775, 34339, 27920, 23402, 19924, 28876, 23483,
        23411,  9230,  5571, 33034, 19926, 34687, 13037,  5848,  7028,
        27054, 23614, 36572, 16929,   686, 30361, 27593,  9483, 22128,
        21026, 34688, 20966, 22690,   707, 32648, 36513,  4266]),
 67: array([12917,  1298,  4129, 37782,   370, 10885,  9652, 33805,  5112,
        37940, 35730, 35795, 35478,  7069,  4242,  1027, 23177,  9643,
        23749, 35408, 30675, 16937, 30083, 13064, 33327, 16684, 33112,
        10321,  7031,  1390, 19066,  9775, 13069, 37436, 37455,   154,
        34573, 29779,  6710, 16780, 37115, 33418, 20930, 27986,  3874,
         8773, 35693, 23259, 1028

In [8]:
def tf(counts):
    tfDF = pd.DataFrame()
    
    for index, row in counts.iterrows():
        tfDF[index] = row.div(row.sum())
        
    return tfDF

tf = pd.DataFrame(tf(wordCounts)).T

In [9]:
idf = tf.apply(lambda x: np.log((df['AGENCY_NUM'].max() + 1) / (np.count_nonzero(x.to_numpy())+1)), axis=0)

In [10]:
prod = tf*idf

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(prod).astype("float")
for i in range(len(similarity)):
    similarity[i][i] = 0

In [12]:
# Find most similar vendors
mostsimilar = np.argmax(similarity, axis=1)
mostsimilar_verbose = np.array([df_agencies.iloc[i][1] for i in mostsimilar])
mostsimilar_vendors = np.array([bags[i] for i in mostsimilar], dtype = object)

In [13]:
grouped = df.groupby(['AGENCY_NUM', 'VENDOR_NAME_NUM'], as_index = False)['TRANSACTION_AMOUNT'].agg('sum')
topn = grouped.sort_values(['AGENCY_NUM','TRANSACTION_AMOUNT'], ascending=False).groupby('AGENCY_NUM').head(5)

In [14]:
descriptions = df.loc[:, ['VENDOR_NAME_NUM', 'TRANSACTION_AMOUNT', 'MCC_DESCRIPTION']]
descriptions['MCC_DESCRIPTION'] = descriptions['MCC_DESCRIPTION'].str.replace('Not Elsewhere Classified', '').str.replace('NOT ELSEWHERE CLASSIFIED', '')
descriptions['MCC_DESCRIPTION_LIST'] = descriptions['MCC_DESCRIPTION'].str.split(r',\s*(?![^()]*\))')

In [15]:
exploded = descriptions.explode('MCC_DESCRIPTION_LIST').replace(' ', np.nan).dropna(0)

In [16]:
grouped_descriptions = exploded.groupby('MCC_DESCRIPTION_LIST').sum('TRANSACTION_AMOUNT')
topn_descriptions = pd.DataFrame(grouped_descriptions['TRANSACTION_AMOUNT'].groupby('MCC_DESCRIPTION_LIST', group_keys = False).nlargest(25))

In [17]:
# TOP N
topn

,AGENCY_NUM,VENDOR_NAME_NUM,TRANSACTION_AMOUNT
48074,88,24206,554789.05
48171,88,29793,221999.31
47991,88,20930,129553.78
48203,88,30791,116204.61
48252,88,32622,103940.03
...,...,...,...
28,0,32622,16490.67
13,0,10283,3341.96
22,0,14975,2050.00
32,0,34339,1982.25


In [18]:
# topn_descriptions
descriptions

,VENDOR_NAME_NUM,TRANSACTION_AMOUNT,MCC_DESCRIPTION,MCC_DESCRIPTION_LIST
0,37719,229.50,"Industrial Supplies,","[Industrial Supplies, ]"
1,6406,3147.33,"Stationery, Office & School Supply Stores","[Stationery, Office & School Supply Stores]"
2,36174,54.00,Motion Pictures/Video Tape Production&Distribu...,[Motion Pictures/Video Tape Production&Distrib...
3,32645,137.35,"Stationery,Office Supplies,Printing/Writing Paper","[Stationery, Office Supplies, Printing/Writing..."
4,12917,1340.80,"Stationery, Office & School Supply Stores","[Stationery, Office & School Supply Stores]"
...,...,...,...,...
390104,4266,59.96,BOOK STORES,[BOOK STORES]
390105,17784,480.00,ELECTRICAL PARTS AND EQUIPMENT,[ELECTRICAL PARTS AND EQUIPMENT]
390106,17784,2112.00,ELECTRICAL PARTS AND EQUIPMENT,[ELECTRICAL PARTS AND EQUIPMENT]
390107,14337,3689.01,COMMERCIAL EQUIPMENT,[COMMERCIAL EQUIPMENT ]


In [49]:
descriptions_per_agency = {}
for i in range(df_agencies.shape[0]):
    top_5_vendors = topn[topn['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].to_numpy()
    
    desc = []
    for j in top_5_vendors:
        desc += np.unique(descriptions[descriptions['VENDOR_NAME_NUM'] == j]['MCC_DESCRIPTION_LIST'].to_numpy()).tolist()[0]
        
    descriptions_per_agency[i] = desc

print(descriptions_per_agency)

{0: ['COMMERCIAL EQUIPMENT  ', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'Non-Durable Goods', '', 'Professional Services ', 'Business Services '], 1: ['COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'COMMERCIAL EQUIPMENT  ', 'Business Services ', 'Commercial Photography', 'Art & Graphics', 'Office and Commercial Furniture'], 2: ['Computer Maint&Repair Service', 'Not Elsewhere Class.', 'BOOK STORES', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'MISCELLANEOUS PUBLISHING AND PRINTING', 'OFFICE  SCHOOL SUPPLY  AND STATIONERY STORES'], 3: ['PROFESSIONAL SERVICES-', 'COMMERCIAL EQUIPMENT  ', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'Accounting', 'Auditing & Bookkeeping Services', 'COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE'], 4: ['COMPUTERS  COMPUTER PERIPHERAL EQUIPMENT  SOFTWARE', 'STATIONERY-OFFICE SUPPLIES-PRINTING + WRITING PAP.', 'OFFICE PHOTOGRAPHIC PHOTOCOPY + MICROFILM EQUIPMT.', 'COMMERCIAL EQUIPMENT  ', 'DISCOUNT STORES'], 5: ['COMP

In [151]:
vendor_names = exploded.loc[:, ['VENDOR_NAME_NUM', 'MCC_DESCRIPTION_LIST']].dropna(0)
vendor_names['MCC_DESCRIPTION_LIST'] = vendor_names['MCC_DESCRIPTION_LIST'].str.upper()
df_vendors.join

def summarize(agency = 0):
    """
    Gives brief information on the agency and its most similar agency.
    """
    
    # gets the list of vendors this agency uses
    agency_bag = bags[agency]
    agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in agency_bag])
        
    # get the vendors of the most similar agency
    similar_agency_bag = bags[mostsimilar[agency]]
    similar_agency_bag_verbose = pd.DataFrame([df_vendors.iloc[i][1] for i in similar_agency_bag]).join(vendor_names)
    
    similar_agency_bag_verbose = similar_agency_bag_verbose
        
    topVendors = [x.strip().upper() for x in descriptions_per_agency[mostsimilar[agency]]]
    
    newVendors = []
    
    for index, row in similar_agency_bag_verbose.iterrows():
        if row['MCC_DESCRIPTION_LIST'] in topVendors:
            if row[0] not in agency_bag_verbose:
                newVendors.append(row[0])
        
#     print("Selected Agency:", df_agencies.iloc[agency][1])
#     print("Most Similar Agency:", mostsimilar_verbose[agency])
#     print()
#     print("Agency Vendors:", agency_bag_verbose)
#     print()
#     print("Most Similar Agency Vendors:", similar_agency_bag_verbose[0].to_numpy())
#     print()
#     print("Potential New Vendors:", newVendors)
    
    summaryDict = {
        'agency': df_agencies.iloc[agency][1],
        'agency_num': int(df_agencies.iloc[agency][0]),
        'similar_agency': mostsimilar_verbose[agency].tolist(),
        'agency_vendors': agency_bag_verbose.tolist(),
        'similar_agency_vendors': similar_agency_bag_verbose[0].to_numpy().tolist(),
        'new_vendors': newVendors.tolist
    }
    
    print(summaryDict)
    
    return summaryDict


In [152]:
bigDict = {}

for i in range(0, 89):
    bigDict[i] = summarize(i)

AttributeError: 'list' object has no attribute 'tolist'

In [150]:
import json

with open("results.json", "w") as outfile:  
    json.dump(bigDict, outfile) 

TypeError: Object of type 'ndarray' is not JSON serializable